# 01 – Data Collection
In this notebook we collect Trustpilot reviews for Allianz Suisse.

In [4]:
import requests, pandas as pd
from bs4 import BeautifulSoup
print("requests:", requests.__version__)
print("pandas:", pd.__version__)

requests: 2.32.4
pandas: 2.3.1


In [5]:
import os, json, time, re
from typing import List, Dict, Any
import requests
import pandas as pd
from bs4 import BeautifulSoup

BASE_URL = "https://ch.trustpilot.com"
BUSINESS_PATH = "/review/allianz.ch"  # page for Allianz Suisse

def _extract_from_next_data(soup: BeautifulSoup) -> List[Dict[str, Any]]:
    """Extract reviews from the Next.js __NEXT_DATA__ JSON if present."""
    out = []
    tag = soup.find("script", id="__NEXT_DATA__", type="application/json")
    if not tag or not tag.string:
        return out
    try:
        data = json.loads(tag.string)
    except Exception:
        return out

    # recursive search for review-like dicts
    def visit(x):
        if isinstance(x, dict):
            # common keys found in Next.js payloads (names vary over time)
            keys = set(x.keys())
            if {"text", "rating", "createdAt"} <= keys:
                out.append({
                    "text": x.get("text"),
                    "date": x.get("createdAt"),
                    "rating": x.get("rating"),
                })
            elif {"reviewText", "rating", "createdAt"} <= keys:
                out.append({
                    "text": x.get("reviewText"),
                    "date": x.get("createdAt"),
                    "rating": x.get("rating"),
                })
            for v in x.values():
                visit(v)
        elif isinstance(x, list):
            for v in x:
                visit(v)

    visit(data)
    # keep only plausible rows
    out = [r for r in out if r.get("text") and (r.get("date") or r.get("rating"))]
    return out

def _extract_from_html(soup: BeautifulSoup) -> List[Dict[str, Any]]:
    """Fallback: parse visible review cards with more specific selectors."""
    out = []
    cards = []
    # common wrappers for review cards (Trustpilot changes classes often)
    cards += soup.select("section[data-service-review-card-paper]")
    cards += soup.select("section[class*='reviewCard'], article[class*='reviewCard']")
    cards = cards or soup.select("article, section")  # very last resort

    for c in cards:
        # 1) review text (prefer dedicated data-attribute, then known classes)
        text_el = (
            c.select_one("p[data-service-review-text-typography]") or
            c.select_one("p[class*='review-content__text']") or
            c.select_one("div[data-service-review-text-typography] p")
        )
        text = text_el.get_text(" ", strip=True) if text_el else None

        # 2) date (ISO usually in datetime attr)
        time_el = c.find("time")
        date = time_el.get("datetime") if time_el else None

        # 3) rating (look for aria-label or data-rating)
        rating = None
        star_el = c.select_one("[aria-label*='star'], [aria-label*='Stern'], [aria-label*='étoile'], [data-rating], img[alt*='star']")
        if star_el:
            label = star_el.get("aria-label") or star_el.get("alt") or ""
            m = re.search(r"(\d+(?:\.\d+)?)", label)  # first number in the label
            if m:
                try:
                    rating = float(m.group(1))
                except Exception:
                    rating = None
        if not rating and star_el and star_el.has_attr("data-rating"):
            try:
                rating = float(star_el["data-rating"])
            except Exception:
                rating = None

        if text and (date or rating is not None):
            out.append({"text": text, "date": date, "rating": rating})
    return out

_CLEAN_PATTERNS = [
    r"^\s*[A-Z]{2,}\w*\s*",                         # initials like "NE" or "WA"
    r"•\s*\d+\s*Bewertungen?",                      # "• 1 Bewertung"
    r"\bVor\s+\d+\s+Tagen\b",                       # "Vor 3 Tagen"
    r"\bAktualisiert am\b.*?$",                     # "Aktualisiert am ..."
]

def _clean_text(t: str) -> str:
    if not t:
        return t
    x = t
    for pat in _CLEAN_PATTERNS:
        x = re.sub(pat, "", x, flags=re.IGNORECASE)
    x = re.sub(r"\s{2,}", " ", x).strip()
    return x

def fetch_trustpilot_reviews(business_path: str, pages: int = 2, sleep_sec: float = 1.0) -> pd.DataFrame:
    rows = []
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome Safari"}
    for page in range(1, pages + 1):
        url = f"{BASE_URL}{business_path}?page={page}"
        resp = requests.get(url, headers=headers, timeout=20)
        if resp.status_code == 404:
            print(f"Page not found (404): {url}")
            break
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # Prefer Next.js JSON if available, else HTML
        part = _extract_from_next_data(soup)
        if not part:
            part = _extract_from_html(soup)

        for r in part:
            r["text"] = _clean_text(r.get("text", ""))
            # normalize date to ISO
            if r.get("date"):
                try:
                    r["date"] = pd.to_datetime(r["date"]).isoformat()
                except Exception:
                    pass
            r["source"] = "trustpilot"
            rows.append(r)

        print(f"Page {page}: collected {len(part)} reviews")
        time.sleep(sleep_sec)
        if not part:
            break

    df = pd.DataFrame(rows, columns=["text", "date", "rating", "source"])
    # drop empty texts and duplicates
    df = df.dropna(subset=["text"]).drop_duplicates(subset=["text", "date"])
    return df

df_tp = fetch_trustpilot_reviews(BUSINESS_PATH, pages=2, sleep_sec=1.0)
df_tp.head(10)


Page 1: collected 20 reviews
Page 2: collected 20 reviews


,text,date,rating,source
0,schlechter Service. Cabriolet verdeck erlitt e...,2025-08-06T15:24:27+00:00,None,trustpilot
1,Service! Besuchstermine werden vergessen Mails...,2025-08-06T09:45:07+00:00,None,trustpilot
2,empfehlenswert – enttäuschende Erfahrung mit d...,2025-07-16T10:16:03+00:00,None,trustpilot
3,Unfallversicherung! Erst wartet man Wochen lan...,2025-07-15T11:14:16+00:00,None,trustpilot
4,"Assistance Europaweit, das Auto stehen geblieb...",2025-06-30T18:37:56+00:00,None,trustpilot
5,". So gar ein Stern zu viel. Sehr Unfreundlich,...",2025-06-26T22:45:30+00:00,None,trustpilot
6,"10,j bin da, Nur Bezahlt!!! Jedes Jahr nur Erh...",2025-06-20T18:27:00+00:00,None,trustpilot
7,wieder Allianz – enttäuschend und unfair! Wir ...,2025-05-18T07:31:43+00:00,None,trustpilot
8,habe mit meiner Kreditkarte einen Einkauf Onli...,2025-04-28T09:55:59+00:00,None,trustpilot
9,"Jahr 2021 eine Offerte erhalten, zugesagt und ...",2025-04-19T21:59:31+00:00,None,trustpilot


In [6]:
# Quick QA
print("Rows, Cols:", df_tp.shape)
print("\nSample:")
display(df_tp.head(5))

print("\nRating distribution (NaNs included):")
print(df_tp["rating"].value_counts(dropna=False))

print("\nText length stats:")
print(df_tp["text"].astype(str).str.len().describe())

Rows, Cols: (40, 4)

Sample:


,text,date,rating,source
0,schlechter Service. Cabriolet verdeck erlitt e...,2025-08-06T15:24:27+00:00,None,trustpilot
1,Service! Besuchstermine werden vergessen Mails...,2025-08-06T09:45:07+00:00,None,trustpilot
2,empfehlenswert – enttäuschende Erfahrung mit d...,2025-07-16T10:16:03+00:00,None,trustpilot
3,Unfallversicherung! Erst wartet man Wochen lan...,2025-07-15T11:14:16+00:00,None,trustpilot
4,"Assistance Europaweit, das Auto stehen geblieb...",2025-06-30T18:37:56+00:00,None,trustpilot



Rating distribution (NaNs included):
rating
None    40
Name: count, dtype: int64

Text length stats:
count      40.000000
mean      546.875000
std       510.379305
min        45.000000
25%       182.500000
50%       368.500000
75%       694.000000
max      2015.000000
Name: text, dtype: float64


In [8]:
from langdetect import detect

def safe_detect(text: str) -> str:
    try:
        return detect(text)
    except Exception:
        return "unk"

# Basic cleaning
df_clean = (
    df_tp
    .dropna(subset=["text"])                       # remove empty texts
    .drop_duplicates(subset=["text","date"])       # deduplicate
    .assign(text=lambda d: d["text"].str.strip())
)

# Keep DE/FR/IT only and remove very short texts (< 20 chars)
df_clean["lang"] = df_clean["text"].astype(str).apply(safe_detect)
df_clean = df_clean[df_clean["lang"].isin(["de","fr","it"])]
df_clean = df_clean[df_clean["text"].str.len() >= 20]

print("Cleaned shape:", df_clean.shape)
display(df_clean.sample(min(5, len(df_clean))))

# Save
import os
os.makedirs("../data/raw", exist_ok=True)
os.makedirs("../data/clean", exist_ok=True)

raw_path   = "../data/raw/trustpilot_allianz.csv"
clean_path = "../data/clean/trustpilot_allianz_clean.csv"

df_tp.to_csv(raw_path, index=False)
df_clean.to_csv(clean_path, index=False)
raw_path, clean_path

df_tp = fetch_trustpilot_reviews(BUSINESS_PATH, pages=10, sleep_sec=1.0)
len(df_tp)

Cleaned shape: (40, 5)


,text,date,rating,source,lang
30,an meinem Auto von Versicherungsnehmer Allianz...,2024-09-02T17:46:27+00:00,None,trustpilot,de
35,63 wegen fehlerhafte Auszahlungen von Krankent...,2024-07-09T08:13:37+00:00,None,trustpilot,de
16,"war immer da ,rasch, unkompliziert,keine aufwe...",2025-01-04T15:08:49+00:00,None,trustpilot,de
4,"Assistance Europaweit, das Auto stehen geblieb...",2025-06-30T18:37:56+00:00,None,trustpilot,de
27,nicht selbst verschuldete Parkschäden links un...,2024-09-23T18:53:21+00:00,None,trustpilot,de


Page 1: collected 20 reviews
Page 2: collected 20 reviews
Page 3: collected 20 reviews
Page 4: collected 20 reviews
Page 5: collected 20 reviews
Page 6: collected 18 reviews
Page 7: collected 18 reviews
Page not found (404): https://ch.trustpilot.com/review/allianz.ch?page=8


136